In [ ]:
# In this notebook CNN is trained to clasify the phases of a system of hard rods on a square lattice

import numpy as np
import pickle

In [ ]:
# path to the data
path98 = '/content/drive/My Drive/rods98_data_correct/'
path126 = '/content/drive/My Drive/rods126_data_correct/'
path154 = '/content/drive/My Drive/rods154_data_correct/'
path182 = '/content/drive/My Drive/rods182_data_correct/'

In [ ]:
# mu_values of the train data
d98 =  [0.5, 0.6, 0.7, 0.8, 1.10, 1.20, 1.30, 1.40]
d126 = [0.5, 0.6, 0.7, 0.8, 0.9, 1.20, 1.30, 1.40, 1.50, 1.60]
d154 = [0.5, 0.6, 0.8, 0.9, 1.0, 1.30, 1.40, 1.50, 1.70, 1.80]
d182 = [0.5, 0.7, 0.8, 0.9, 1.0, 1.30, 1.40, 1.50, 1.60, 1.70]

In [ ]:
# raw data to matrices (train data)
def data_to_lattice(path,L,d):
  lattice = np.zeros((15000,int(L),int(L)))
  label = np.zeros((15000,1))
  j = 0
  for i in d:
    datafile = open(path+'snapshot'+str("{:.2f}".format(i))+'.pickle','rb')
    data = pickle.load(datafile)
    datafile.close()
    lattice[j:j+1500] +=data
    del data
    datafile = open(path+'label'+str("{:.2f}".format(i))+'.pickle','rb')
    labels = pickle.load(datafile)
    datafile.close()
    label[j:j+1500] += labels
    j = j+1500
    print('mc=',"{:.2f}".format(i),"done")

  return lattice, label

In [ ]:
#loading data

lattice, label = data_to_lattice(path98, 98, d98)
lattice126, label126 = data_to_lattice(path126, 126, d126)
lattice154, label154 = data_to_lattice(path154, 154, d154)
lattice182, label182 = data_to_lattice(path182, 182, d182)

In [ ]:
# importing libraries

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Conv2D, MaxPooling2D, Dropout
import keras

In [ ]:
# test_train_split 
# in this notebook training on the data of size L = 154

X_train,X_test,Y_train,Y_test = train_test_split(lattice154,label154,test_size=0.15)

X_train = np.expand_dims(X_train,axis=3)
X_test = np.expand_dims(X_test,axis=3)

del lattice154 # to erase unwanted data

In [ ]:
# CNN architecture

model = Sequential()
model.add(Conv2D(filters=4,input_shape=(154,154,1),kernel_size=(3,3),activation='relu',kernel_regularizer=keras.regularizers.l2(0.1)))#previously l2=0.05
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=4,kernel_size=(3,3),activation='relu',kernel_regularizer=keras.regularizers.l2(0.075)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Flatten())
model.add(Dropout(0.3)) #used to be 0.3 for rods first case
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
# model summary
model.summary()

In [ ]:
# compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# fitting the data
model.fit(X_train,Y_train,validation_split=0.15,batch_size=64,epochs=100) 

Testing 

In [ ]:
# raw data to matrices (test)
def testdata_to_lattice(path,L,d):
  lattice = np.zeros((6000,int(L),int(L)))
  label = np.zeros((6000,1))
  j = 0
  for i in d:
    datafile = open(path+'snapshot'+str("{:.4f}".format(i))+'.pickle','rb')
    data = pickle.load(datafile)
    datafile.close()
    lattice[j:j+1000] +=data
    del data
    datafile = open(path+'label'+str("{:.4f}".format(i))+'.pickle','rb')
    labels = pickle.load(datafile)
    datafile.close()
    label[j:j+1000] += labels
    j = j+1000
    print('mc=',"{:.2f}".format(i),"done")

  return lattice, label

In [ ]:
# test path
test_path98 = '/content/drive/My Drive/rods98_data_correct/test/'
test_path126 = '/content/drive/My Drive/rods126_data_correct/test/'
test_path154 = '/content/drive/My Drive/rods154_data_correct/test/'

# test mu_values
te98 =  [0.8245, 0.8730, 0.9215, 0.9458, 0.9942, 1.0185, 1.0670, 1.1155]
te126 = [0.8925, 0.9450, 0.9975, 1.0238, 1.0763, 1.1025, 1.1550, 1.2075]
te154 = [0.9435, 0.9990, 1.0545, 1.0823, 1.1378, 1.1655, 1.2210, 1.2765]
te182 = [0.9690, 1.0260, 1.0830, 1.1115, 1.1685, 1.1970, 1.2540, 1.3110]

In [ ]:
# loading the data
t98, l98 = testdata_to_lattice(test_path98,98,te98)
t126, l126 = testdata_to_lattice(test_path126,126,te126)
t154, l154 = testdata_to_lattice(test_path154,154,te154)
t182, l182 = testdata_to_lattice(test_path182,182,te182)

In [ ]:
# testing the CNN
i = 0
while(i<1600):
  print(model.evaluate(t154[i:i+200],l154[i:i+200]))
  i += 200